In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from src.web_tasks.page_summary import WebsiteContentSummarizer
from rich.markdown import Markdown
from rich.console import Console
from src.config import OPENAIVARS

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
openai = OpenAI()
openai_model = OPENAIVARS.open_ai_model

### Surpress Logs

In [ ]:
import logging

#Surpresses all logs below critical
logging.getLogger().setLevel(logging.CRITICAL)

# Restores logging
# logging.getLogger().setLevel(logging.DEBUG) 

### Prompts

In [42]:
system_prompt = (
    "You are a highly skilled **business analyst assistant**, specializing in analyzing website content and generating structured, detailed, and insightful business reports.\n\n"

    "**🔹 Your Responsibilities:**\n"
    "- Provide an in-depth **business analysis report** based strictly on the provided website content.\n"
    "- **Extract insights**, rather than simply summarizing the text.\n"
    "- Ignore **navigation elements** (menus, footers, sidebars, repetitive content).\n"
    "- Focus on **core business information** (services, products, mission, vision, customer engagement strategies, competitive positioning, upcoming events, etc.).\n"
    "- Highlight **unique business aspects** and **opportunities for growth**.\n"
    "- Identify **target demographics** who would benefit from the website’s offerings.\n\n"

    "**🔹 Response Format (STRICT Markdown Structure Required):**\n"
    "- The report **must be in Markdown** for readability.\n"
    "- Use **clear headings, bullet points, and structured content**.\n"
    "- If the website provides event details, include a structured table with event names, dates, and target audiences.\n"
    "- Incorporate **quotes or testimonials** from the website (if available) for added credibility.\n\n"

    "**🔹 Important Notes:**\n"
    "- Do not fabricate information—only analyze and interpret what is available.\n"
    "- Use structured Markdown formatting for better readability.\n"
    

)



user_prompt = (

    "**WEBSITE CONTENT PROVIDED BELOW:**\n\n"

    "**🔹 TASK:**\n"
    "Generate a **comprehensive and structured business analysis report** based strictly on the provided website content.\n\n"

    "**🔹 RESPONSE FORMAT REQUIREMENT:**\n"
    "- The report **must be written in Markdown format**.\n"
    "- Use **structured sections, tables (if applicable), and bullet points**.\n"
    "- Ensure clarity, depth, and proper segmentation of insights.\n\n"

    "**🔹 FINAL TOUCH:**\n"
    "Provide **a summary with key takeaways**, ensuring all insights are well-organized and actionable.\n\n"

    "**WEBSITE CONTENT STARTS BELOW:**"
)




links_filter_system_prompt = (
    "You are an **intelligent web assistant**, responsible for **analyzing a list of hyperlinks** extracted from a website.\n"
    "Your task is to filter and retain only the **most relevant and informative links**, ensuring that the extracted content aligns with the business analysis process.\n\n"

    "**🔹 Purpose & Context:**\n"
    "The selected links will be used by the **business analysis assistant** to generate a comprehensive business report.\n"
    "- `system_prompt`: Guides the assistant in summarizing a company's key offerings based on website content.\n"
    "- `user_prompt`: Instructs the assistant to generate an insightful business report focused on services, products, and competitive positioning.\n\n"

    "**🔹 Link Selection Criteria:**\n"
    "Only keep links that provide information crucial to understanding the **business, its offerings, and unique value proposition**"
    "This includes, but is NOT limited to. :\n"
    "- **About Us** (mission, history, brand identity)\n"
    "- **Products & Services** (detailed breakdown of offerings)\n"
    "- **Blog & News** (business insights, industry trends, company updates)\n"
    "- **Case Studies & Testimonials** (customer success stories, credibility indicators)\n"
    "- **Careers** (team culture, hiring insights, employer brand perception)\n\n"

    "**🔹 Exclude the following links:**\n"
    "- **Navigation or duplicate content** (e.g., menus, footers, repeated elements)\n"
    "- **Social media pages** (Facebook, Instagram, Twitter, LinkedIn, YouTube, etc.)\n"
    "- **Legal & policy pages** (Terms of Service, Privacy Policy, Cookie Policy)\n"
    "- **Contact pages that only contain an email form**\n"
    "- **External third-party websites**, unless they are integral to the business (e.g., official partner integrations)\n"
    "- **Generic FAQ pages that do not provide unique insights about the business**\n"
    "- **Sitemap, login pages, or any technical pages unrelated to business insights**\n\n"

    "**Response Format:**\n"
    "Provide a structured JSON response containing only the retained links with their type classification.\n"
    "Example output:\n"
    "{\n"
    "    'links': [\n"
    "        {'type': 'about', 'url': 'https://company.com/about'},\n"
    "        {'type': 'careers', 'url': 'https://company.com/careers'},\n"
    "        {'type': 'blog', 'url': 'https://company.com/blog'}\n"
    "    ]\n"
    "}"
)


links_filter_user_prompt = "Website: {url}\n\nHere is a list of links found on the website:\n" + "\n"

### Single Page Summary

In [43]:
summariser = WebsiteContentSummarizer("https://dirttrackriders.co.uk/about",openai,openai_model, explore_multiple_links=False,verbosity=True)

summary = summariser.summarize((system_prompt,user_prompt))
console = Console()
console.print(Markdown(summary))

==================== Title Page and Content Page/s ====================

(
    'Learn about Dirt Track racing in the UK. Flat Track',
    'top of page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to
navigate through the menu items.\nABOUT\n"Good fun, fast racing, great people, increased my riding skills. Can\'t 
wait for the season to start"\n- Rick Bearcroft, DTRA Vintage Class Rider\n"The DTRA\xa0is such a welcoming club. 
I\'ve made loads of friends across the country"\n- Darren Legg, DTRA Vintage Class Rider\nDTRA FOUNDERS | Anthony 
Brown & Anna Butler\n\u200b\nCONTACT THE DTRA\nSince 2005 the UK has had its very own flat track championships. The
UK scene has been growing every year, with more new bikes and riders starting to race. Racing in the UK takes place
mainly on speedway tracks, with some use of larger horse trotting tracks and TT courses. Races are normally 
contested between 12 riders in three lines of four riders. The duration of each race is six or eight laps, with 
finals of 12 or 15 laps. Unlike in speedway, flat track bikes have a rear brake and gears.\nOur Pro class attracts 
the cream of UK Flattrack racers and often a handful of guest stars from other disciplines. Our Pro riders are of a
quality that they can compete at an international standard, and do a fantastic job representing the UK in Europe 
and the USA. The cost of DTRA club membership and race entries are kept to a minimum to encourage participation in 
the sport. Many current club riders enjoy the fact that they can race a whole season on a single rear tyre, with 
minimum additional costs.\nAll spectators and riders are welcome at our friendly race meetings, and often our races
are free to spectate. Check out the\n\'ENTER A RACE\'\npage for info on the 2025 calendar.\n© 2025 by Dirt Track 
Riders Association // See You At The Races since 2013\nbottom of page'
)

==================== User Prompt ====================

You are looking at a website titled 'Learn about Dirt Track racing in the UK. Flat Track'.

**WEBSITE CONTENT PROVIDED BELOW:**

**🔹 TASK:**
Generate a **comprehensive and structured business analysis report** based strictly on the provided website 
content.

**🔹 RESPONSE FORMAT REQUIREMENT:**
- The report **must be written in Markdown format**.
- Use **structured sections, tables (if applicable), and bullet points**.
- Ensure clarity, depth, and proper segmentation of insights.

**🔹 FINAL TOUCH:**
Provide **a summary with key takeaways**, ensuring all insights are well-organized and actionable.

**WEBSITE CONTENT STARTS BELOW:**

top of page
HOME
ABOUT
INFO
NEW RIDERS
ENTER A RACE
REGULATIONS
EUROS
PARTNERS
BLOG
More
Use tab to navigate through the menu items.
ABOUT
"Good fun, fast racing, great people, increased my riding skills. Can't wait for the season to start"
- Rick Bearcroft, DTRA Vintage Class Rider
"The DTRA is such a welcoming club. I've made loads of friends across the country"
- Darren Legg, DTRA Vintage Class Rider
DTRA FOUNDERS | Anthony Brown & Anna Butler
​
CONTACT THE DTRA
Since 2005 the UK has had its very own flat track championships. The UK scene has been growing every year, with 
more new bikes and riders starting to race. Racing in the UK takes place mainly on speedway tracks, with some use 
of larger horse trotting tracks and TT courses. Races are normally contested between 12 riders in three lines of 
four riders. The duration of each race is six or eight laps, with finals of 12 or 15 laps. Unlike in speedway, flat
track bikes have a rear brake and gears.
Our Pro class attracts the cream of UK Flattrack racers and often a handful of guest stars from other disciplines. 
Our Pro riders are of a quality that they can compete at an international standard, and do a fantastic job 
representing the UK in Europe and the USA. The cost of DTRA club membership and race entries are kept to a minimum 
to encourage participation in the sport. Many current club riders enjoy the fact that they can race a whole season 
on a single rear tyre, with minimum additional costs.
All spectators and riders are welcome at our friendly race meetings, and often our races are free to spectate. 
Check out the
'ENTER A RACE'
page for info on the 2025 calendar.
© 2025 by Dirt Track Riders Association // See You At The Races since 2013
bottom of page

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Business Analysis Report: Learn about Dirt Track Racing in the UK - Flat Track                  ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                               1. Business Overview                                                

The Dirt Track Riders Association (DTRA) is a governing body that has been promoting flat track racing in the UK   
since 2005. The organization aims to foster a welcoming environment for both new and experienced riders while      
maintaining a high standard of competition. The DTRA emphasizes affordability and accessibility in the sport,      
making it conducive for participation across various demographics.                                                 

                                                      Mission                                                      

 • Promote Flat Track Racing: The DTRA is committed to enhancing the flat track racing scene in the UK, encouraging
   participation from new riders and maintaining competitive standards for seasoned athletes.                      

                                                      Vision                                                       

 • Inclusive Participation: Creating an inclusive community that supports both riders and spectators, thereby      
   nurturing friendships and enhancing racing skills.                                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                 2. Core Offerings                                                 

 • Flat Track Racing Events: Main races occur mainly on speedway tracks and occasionally on larger horse trotting  
   tracks.                                                                                                         
 • Membership: DTRA offers club memberships at minimal costs to encourage broader audience participation.          
 • Racing Classes: Includes a Pro class that features elite UK flat track racers and instances of guest riders from
   other disciplines.                                                                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                              3. Target Demographics                                               

 • New Riders: Individuals interested in pursuing flat track racing, especially beginners looking for a supportive 
   environment.                                                                                                    
 • Experienced Riders: Seasoned racers looking for competitive opportunities and community engagement.             
 • Spectators: Racing fans, families, and communities interested in attending free races and engaging in local     
   motorsport events.                                                                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                         4. Customer Engagement Strategies                                         

 • Community Focus: The testimonials from current riders highlight a strong community spirit and camaraderie. This 
   welcoming approach enhances customer loyalty and encourages word-of-mouth promotion.                            
 • Affordability: By keeping membership fees and race entry costs low, the DTRA makes it easier for new riders to  
   join, which can increase participation.                                                                         
 • Accessibility of Events: Most racing 

### Multiple Page Summary

In [44]:
summariser = WebsiteContentSummarizer("https://dirttrackriders.co.uk",openai,openai_model, explore_multiple_links=True,verbosity=True)

summary = summariser.summarize((system_prompt,user_prompt),(links_filter_system_prompt,links_filter_user_prompt))
console = Console()
console.print(Markdown(summary))

==================== Weblinks Extracted ====================

[
    'https://www.dirttrackriders.co.uk/blog',
    'https://www.instagram.com/dirttrackriders/?hl=en',
    'https://www.youtube.com/channel/UCLJgacR2rOImjpWiFZa4erg',
    'https://www.dirttrackriders.co.uk/euros',
    'https://www.dirttrackriders.co.uk/enter-a-race',
    'https://www.dirttrackriders.co.uk',
    'https://www.dirttrackriders.co.uk/partners',
    'https://www.dirttrackriders.co.uk/about',
    'https://www.dirttrackriders.co.uk/regulations',
    'https://www.dirttrackriders.co.uk/info',
    'https://www.facebook.com/DirtTrackRidersAssociationUK/',
    'https://www.dirttrackriders.co.uk/new-riders'
]

==================== Relevant Links ====================

[
    {'type': 'about', 'url': 'https://www.dirttrackriders.co.uk/about'},
    {'type': 'blog', 'url': 'https://www.dirttrackriders.co.uk/blog'},
    {'type': 'products & services', 'url': 'https://www.dirttrackriders.co.uk/enter-a-race'},
    {'type': 'information', 'url': 'https://www.dirttrackriders.co.uk/info'},
    {'type': 'partners', 'url': 'https://www.dirttrackriders.co.uk/partners'},
    {'type': 'regulations', 'url': 'https://www.dirttrackriders.co.uk/regulations'},
    {'type': 'new riders', 'url': 'https://www.dirttrackriders.co.uk/new-riders'}
]

==================== Title Page and Content Page/s ====================

(
    'Dirt Track Riders Asscoation the home of Dirt Track racing in the UK',
    [
        {
            'type': 'about',
            'title': 'Learn about Dirt Track racing in the UK. Flat Track',
            'content': 'top of page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A 
RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate through the menu items.\nABOUT\n"Good fun, fast
racing, great people, increased my riding skills. Can\'t wait for the season to start"\n- Rick Bearcroft, DTRA 
Vintage Class Rider\n"The DTRA\xa0is such a welcoming club. I\'ve made loads of friends across the country"\n- 
Darren Legg, DTRA Vintage Class Rider\nDTRA FOUNDERS | Anthony Brown & Anna Butler\n\u200b\nCONTACT THE DTRA\nSince
2005 the UK has had its very own flat track championships. The UK scene has been growing every year, with more new 
bikes and riders starting to race. Racing in the UK takes place mainly on speedway tracks, with some use of larger 
horse trotting tracks and TT courses. Races are normally contested between 12 riders in three lines of four riders.
The duration of each race is six or eight laps, with finals of 12 or 15 laps. Unlike in speedway, flat track bikes 
have a rear brake and gears.\nOur Pro class attracts the cream of UK Flattrack racers and often a handful of guest 
stars from other disciplines. Our Pro riders are of a quality that they can compete at an international standard, 
and do a fantastic job representing the UK in Europe and the USA. The cost of DTRA club membership and race entries
are kept to a minimum to encourage participation in the sport. Many current club riders enjoy the fact that they 
can race a whole season on a single rear tyre, with minimum additional costs.\nAll spectators and riders are 
welcome at our friendly race meetings, and often our races are free to spectate. Check out the\n\'ENTER A 
RACE\'\npage for info on the 2025 calendar.\n© 2025 by Dirt Track Riders Association // See You At The Races since 
2013\nbottom of page'
        },
        {
            'type': 'blog',
            'title': 'BLOG | Dirt Track Riders',
            'content': "top of page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A 
RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate through the menu items.\nAll Posts\nRace 
Previews\nReports\nSearch\nLog in / Sign up\nalexraby\nApr 16, 2023\n3 min read\n2023 DTRA RD1 GREENFIELD OVAL 
[REPORT]\nBirtwistle makes successful start to title defence Reigning DTRA UK Flat Track National Champion Gary 
Birtwistle opened the 2023 season...\n258 views\n0 comments\nPost not marked as liked\nusefultone\nSep 7, 2021\n4 
min read\nRace Preview: DTRA Nationals Round 6 & Hooligans Round 4 2021, Greenfield Dirt Track TT\nIt’s hard to 
believe that this year’s DTRA flat track racing season is almost over, but it’s time to get ready for our sixth and
final...\n807 views\n0 comments\nPost not marked as liked\nalexraby\nAug 17, 2021\n3 min read\n2021 DTRA RD4 KING'S
LYNN [REPORT]\nHALES TAKES COMMAND Toby Hales took a major step towards recovering the Indian Motorcycle DTRA UK 
Flat Track Nationals title he last won...\n215 views\n0 comments\nPost not marked as liked\nusefultone\nAug 8, 
2021\n3 min read\nRace Preview: DTRA Nationals Round 4 & Hooligans Round 3 2021, Adrian Flux Arena, King’s 
Lynn\nThe DTRA is heading back to Adrian Flux Arena in King’s Lynn, Norfolk for the next round of flat track racing
on Sunday 15th August.\n249 views\n0 comments\nPost not marked as liked\n© 2025 by Dirt Track Riders Association //
See You At The Races since 2013\nbottom of page"
        },
        {
            'type': 'products & services',
            'title': 'ENTER A RACE | Dirt Track Riders',
            'content': 'top of page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A 
RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate through the menu items.\n2025\nSCHEDULE\nRACE 
SCHEDULE 2025\nDTRA UK Flattrack Nationals\n\u200b\nThe 2025 season is hotting up we are s

==================== User Prompt ====================

You are looking at a website titled 'Dirt Track Riders Asscoation the home of Dirt Track racing in the UK'.

**WEBSITE CONTENT PROVIDED BELOW:**

**🔹 TASK:**
Generate a **comprehensive and structured business analysis report** based strictly on the provided website 
content.

**🔹 RESPONSE FORMAT REQUIREMENT:**
- The report **must be written in Markdown format**.
- Use **structured sections, tables (if applicable), and bullet points**.
- Ensure clarity, depth, and proper segmentation of insights.

**🔹 FINAL TOUCH:**
Provide **a summary with key takeaways**, ensuring all insights are well-organized and actionable.

**WEBSITE CONTENT STARTS BELOW:**

[{'type': 'about', 'title': 'Learn about Dirt Track racing in the UK. Flat Track', 'content': 'top of 
page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate 
through the menu items.\nABOUT\n"Good fun, fast racing, great people, increased my riding skills. Can\'t wait for 
the season to start"\n- Rick Bearcroft, DTRA Vintage Class Rider\n"The DTRA\xa0is such a welcoming club. I\'ve made
loads of friends across the country"\n- Darren Legg, DTRA Vintage Class Rider\nDTRA FOUNDERS | Anthony Brown & Anna
Butler\n\u200b\nCONTACT THE DTRA\nSince 2005 the UK has had its very own flat track championships. The UK scene has
been growing every year, with more new bikes and riders starting to race. Racing in the UK takes place mainly on 
speedway tracks, with some use of larger horse trotting tracks and TT courses. Races are normally contested between
12 riders in three lines of four riders. The duration of each race is six or eight laps, with finals of 12 or 15 
laps. Unlike in speedway, flat track bikes have a rear brake and gears.\nOur Pro class attracts the cream of UK 
Flattrack racers and often a handful of guest stars from other disciplines. Our Pro riders are of a quality that 
they can compete at an international standard, and do a fantastic job representing the UK in Europe and the USA. 
The cost of DTRA club membership and race entries are kept to a minimum to encourage participation in the sport. 
Many current club riders enjoy the fact that they can race a whole season on a single rear tyre, with minimum 
additional costs.\nAll spectators and riders are welcome at our friendly race meetings, and often our races are 
free to spectate. Check out the\n\'ENTER A RACE\'\npage for info on the 2025 calendar.\n© 2025 by Dirt Track Riders
Association // See You At The Races since 2013\nbottom of page'}, {'type': 'blog', 'title': 'BLOG | Dirt Track 
Riders', 'content': "top of page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A 
RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate through the menu items.\nAll Posts\nRace 
Previews\nReports\nSearch\nLog in / Sign up\nalexraby\nApr 16, 2023\n3 min read\n2023 DTRA RD1 GREENFIELD OVAL 
[REPORT]\nBirtwistle makes successful start to title defence Reigning DTRA UK Flat Track National Champion Gary 
Birtwistle opened the 2023 season...\n258 views\n0 comments\nPost not marked as liked\nusefultone\nSep 7, 2021\n4 
min read\nRace Preview: DTRA Nationals Round 6 & Hooligans Round 4 2021, Greenfield Dirt Track TT\nIt’s hard to 
believe that this year’s DTRA flat track racing season is almost over, but it’s time to get ready for our sixth and
final...\n807 views\n0 comments\nPost not marked as liked\nalexraby\nAug 17, 2021\n3 min read\n2021 DTRA RD4 KING'S
LYNN [REPORT]\nHALES TAKES COMMAND Toby Hales took a major step towards recovering the Indian Motorcycle DTRA UK 
Flat Track Nationals title he last won...\n215 views\n0 comments\nPost not marked as liked\nusefultone\nAug 8, 
2021\n3 min read\nRace Preview: DTRA Nationals Round 4 & Hooligans Round 3 2021, Adrian Flux Arena, King’s 
Lynn\nThe DTRA is heading back to Adrian Flux Arena in King’s Lynn, Norfolk for the next round of flat track racing
on Sunday 15th August.\n249 views\n0 comments\nPost not marked as liked\n© 2025 by Dirt Track 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                         Business Analysis Report: Dirt Track Riders Association (DTRA)                          ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                                 Executive Summary                                                 

The Dirt Track Riders Association (DTRA) serves as the leading organization for flat track racing in the UK.       
Established in 2005, DTRA facilitates and promotes flat track racing, focusing on community engagement and         
accessibility. The organization offers a structured framework for racing, ensuring that participants of all        
levels—from newcomers to seasoned racers—can compete effectively.                                                  


                                              Core Business Insights                                               

                                                Mission and Vision                                                 

 • Mission: To provide a welcoming environment for all riders, ensuring a fun and competitive flat track racing    
   experience.                                                                                                     
 • Vision: To elevate flat track racing in the UK, promoting growth in participation and enhancing the sport's     
   visibility.                                                                                                     

                                               Products and Services                                               

 • Race Participation: DTRA organizes races nationwide, primarily on speedway and trotting tracks.                 
 • Membership: Open for individuals interested in racing, with fees designed to be affordable to encourage         
   participation (Adult membership: £40, Youth membership: £30).                                                   
 • Racing Classes: Various classes available for all ages and skill levels, including:                             
    • Pro Class                                                                                                    
    • Rookie Class                                                                                                 
    • Youth Classes                                                                                                
    • Vintage Classes                                                                                              
 • Event Calendar: Structured race schedule, with 7 National Championship rounds planned for 2025.                 

                                          Customer Engagement Strategies                                           

 • Community-building: Cultivating a friendly atmosphere during race events where both spectators and riders are   
   welcome.                                                                                                        
 • Support for New Riders: Providing detailed guidance and resources for newcomers, including riding schools and a 
   wealth of information on participation.                                                                         
 • Testimonials: Positive feedback from members highlights the communal aspect; for example:                       
   ▌ “The DTRA is such a welcoming club. I’ve made loads of friends across the country.” - Darren Legg, DTRA       
   ▌ Vintage Class Rider.                                                                                          

                                              Unique Business Aspects                                              

 • Cost-effective participation: Racing costs are minimized (e.g., racers can complete a season on a single rear   
   tire) to enhance accessibility.         